In [ ]:
!pip install transformers tokenizers datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
input_filename = "naslovi_new.input"
dataset_filename = input_filename + '.prepared'
pretrained_model_name = 'gpt2'
new_model_name = pretrained_model_name + '_5'

In [6]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="/content/drive/MyDrive/ML-Vesti/naslovi-tokenizer.json")
tokenizer.pad_token = "<pad>"


In [7]:
from datasets import load_dataset
# datasets = load_dataset('text', data_files=dataset_filename, split="train[90%],validate")
eval, train = load_dataset('text', data_files='/content/drive/MyDrive/ML-Vesti/' + dataset_filename, split=['train[:10%]', 'train[10%:]'])
def prepare_for_trainer(example):
  tokenized = tokenizer(example['text'], truncation=True, padding='max_length', max_length=20)
  tokenized["labels"] = tokenized["input_ids"].copy()
  return tokenized
tokenized_train = train.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])
tokenized_eval = eval.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])


Using custom data configuration default-9b16dadd29989af1


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-9b16dadd29989af1/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.
 

In [8]:
from transformers import Trainer, AutoConfig, AutoModelForCausalLM, TrainingArguments

config = AutoConfig.from_pretrained(pretrained_model_name)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name)
# model.to('cuda:0')
training_args = TrainingArguments(
    # new_model_name,
    output_dir='/content/drive/MyDrive/ML-Vesti/' + new_model_name,
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    save_strategy='epoch',
    num_train_epochs=7,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    # prediction_loss_only=True,
    tokenizer=tokenizer,
)


In [9]:
import logging
logger = logging.getLogger(__name__)
logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )

Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False


In [ ]:
# import gc
# import torch 
# model = None
# trainer = None
# gc.collect()
# torch.cuda.empty_cache()

In [10]:
train_result = trainer.train()
trainer.save_model('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)

***** Running training *****
  Num examples = 1968703
  Num Epochs = 7
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 107667


Epoch,Training Loss,Validation Loss
1,4.189200,4.804270
2,3.849300,4.414730
3,3.677400,4.227653
4,3.569300,4.118976
5,3.511100,4.056015
6,3.470400,4.017878
7,3.454200,4.003893


***** Running Evaluation *****
  Num examples = 218745
  Batch size = 128
Saving model checkpoint to /content/drive/MyDrive/ML-Vesti/gpt2_5/checkpoint-15381
Configuration saved in /content/drive/MyDrive/ML-Vesti/gpt2_5/checkpoint-15381/config.json
Model weights saved in /content/drive/MyDrive/ML-Vesti/gpt2_5/checkpoint-15381/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML-Vesti/gpt2_5/checkpoint-15381/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ML-Vesti/gpt2_5/checkpoint-15381/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 218745
  Batch size = 128
Saving model checkpoint to /content/drive/MyDrive/ML-Vesti/gpt2_5/checkpoint-30762
Configuration saved in /content/drive/MyDrive/ML-Vesti/gpt2_5/checkpoint-30762/config.json
Model weights saved in /content/drive/MyDrive/ML-Vesti/gpt2_5/checkpoint-30762/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML-Vesti/gpt2_5/checkpoint-30762/token

In [13]:
trainer.save_model('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)

Saving model checkpoint to /content/drive/MyDrive/ML-Vesti/modeli/gpt2_5
Configuration saved in /content/drive/MyDrive/ML-Vesti/modeli/gpt2_5/config.json
Model weights saved in /content/drive/MyDrive/ML-Vesti/modeli/gpt2_5/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML-Vesti/modeli/gpt2_5/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ML-Vesti/modeli/gpt2_5/special_tokens_map.json


In [11]:
import numpy as np
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)
model.config.pad_token_id = 1

for temp in np.linspace(0.2, 2, 10):
    print()
    print("###### Temperature: " + str(temp))
    for i in range(0, 5):
        out = model.generate(do_sample=True, temperature=temp)[0]
        print(tokenizer.decode(out, skip_special_tokens=True))

loading configuration file /content/drive/MyDrive/ML-Vesti/modeli/gpt2_5/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.8.2",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights file /content/drive/MyDrive/ML-Vesti/mode


###### Temperature: 0.2
Evo šta je sve potrebno da bi se u Srbiji svi voleli
U Srbiji se nalazi jedna od najvećih svetskih destinacijain i
Evo kako je izgledala pre 10 godina!in u Srbiji
Evo šta je rekao o tomein od ponedeljka
Evo kako je izgledala pre 10 godina!in za sve koji

###### Temperature: 0.4
Ovo je pet albuma koje bi trebalo da preslušate danas (VIDEO)
Ovo je prava istina o tome kako su u Srbiji se zarazili
U Srbiji se ne preporučuje da se ne bi ni primećen
Ovo je prava istina o njihovom odnosu!in: Da li je
Ovo je bio prvi, a sada je dobio i jednu od najvećih!

###### Temperature: 0.6000000000000001
Evo šta je Darko Lazić rekao o Đokovićujte
Posle ovog snimka, ovaj snimak je dobio ime! (VIDEO)
Ne može da izdrži, ali se nije pojavioin i
Na granici sa Hrvatskom se čeka i do 20 minuta! (VIDEO)
Vučić u Srbiji: Reprezentativac u borbi protiv terorizmami

###### Temperature: 0.8
Ne žele ni da sanja da će se Vučić i Rusija boriti na izbore
Kad je saznala da se nije čula, imala je i

In [12]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-832f013c-d009-7915-e527-8f6e22855447)
